# About this notebook
- Deberta-base starter code
- pip wheels is [here](https://www.kaggle.com/yasufuminakama/nbme-pip-wheels)
- Inference notebook is [here](https://www.kaggle.com/yasufuminakama/nbme-deberta-base-baseline-inference)

If this notebook is helpful, feel free to upvote :)

# Directory settings

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# CFG

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=False 
    competition='NBME'
    _wandb_kernel='nakama'
    debug=False
    apex=True
    print_freq=100
    num_workers=6
#     model='roberta-large'
#     model="microsoft/deberta-large"
#     model="microsoft/deberta-base"
#     model="ZZ99/NBME_TAPT_deberta_base"
#     tokenizer_model="microsoft/deberta-base"
#     model="ZZ99/NBME_TAPT_deberta_base"
    tokenizer_model="microsoft/deberta-v3-base"
#     tokenizer_model="../input/deberta-v3-base"  
    model="ZZ99/tapt_nbme_deberta_v3_base"
#     model="emilyalsentzer/Bio_ClinicalBERT"
#     model="Tsubasaz/clinical-pubmed-bert-base-128"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=12
    fc_dropout=0.2
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
#     n_fold=2
#     trn_fold=[0, 1]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    
    import wandb

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret_value_0 = user_secrets.get_secret("wandb_api")
        wandb.login(key=secret_value_0)
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='NBME-Public', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

# Library

In [ ]:
# ====================================================
# Library
# Install required dependencies
# ====================================================
import os
import gc
import re
import ast
#abstract syntax tree
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

# os.system('pip uninstall -y transformers')
# os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")

input_dir = Path("../input/d/datasets/nbroad/deberta-v2-3-fast-tokenizer")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()
    print("conversion_path unlinked")
shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in [
    "tokenization_deberta_v2.py",
    "tokenization_deberta_v2_fast.py",
    "deberta__init__.py",
]:
    if str(filename).startswith("deberta"):
        filepath = deberta_v2_path / str(filename).replace("deberta", "")
    else:
        filepath = deberta_v2_path / filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir / filename, filepath)
import tokenizers
import transformers 
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# ====================================================
# tokenizer
# ====================================================
if (
    "v2" in CFG.model
    or "v3" in CFG.model
):
    from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast
    tokenizer = DebertaV2TokenizerFast.from_pretrained(CFG.tokenizer_model)
else: 
    tokenizer = AutoTokenizer.from_pretrained(CFG.model)
# tokenizer = AutoTokenizer.from_pretrained(CFG.tokenizer_model)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

# Helper functions for scoring

In [ ]:
# From https://www.kaggle.com/theoviel/evaluation-metric-folds-baseline

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

In [ ]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    #df.values returns values in numpy form
    for location_list in df['location_for_create_labels'].values:
        truth = []
        #if non-empty
        if len(location_list) > 0:
            location = location_list[0]
            #['0 1;3 4']-> ['0 1','3 4']
            
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results

# Difficulty: How to turn separate word probability into a branch of words as output?
# char_probs shape= set size*history note word size
def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
#return f1 score
def get_score(y_true, y_pred):
    score = span_micro_f1(y_true, y_pred)
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# Data Loading

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

In [ ]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

In [ ]:
# incorrect annotation
train.loc[338, 'annotation'] = ast.literal_eval('[["father heart attack"]]')
train.loc[338, 'location'] = ast.literal_eval('[["764 783"]]')

train.loc[621, 'annotation'] = ast.literal_eval('[["for the last 2-3 months"]]')
train.loc[621, 'location'] = ast.literal_eval('[["77 100"]]')

train.loc[655, 'annotation'] = ast.literal_eval('[["no heat intolerance"], ["no cold intolerance"]]')
train.loc[655, 'location'] = ast.literal_eval('[["285 292;301 312"], ["285 287;296 312"]]')

train.loc[1262, 'annotation'] = ast.literal_eval('[["mother thyroid problem"]]')
train.loc[1262, 'location'] = ast.literal_eval('[["551 557;565 580"]]')

train.loc[1265, 'annotation'] = ast.literal_eval('[[\'felt like he was going to "pass out"\']]')
train.loc[1265, 'location'] = ast.literal_eval('[["131 135;181 212"]]')

train.loc[1396, 'annotation'] = ast.literal_eval('[["stool , with no blood"]]')
train.loc[1396, 'location'] = ast.literal_eval('[["259 280"]]')

train.loc[1591, 'annotation'] = ast.literal_eval('[["diarrhoe non blooody"]]')
train.loc[1591, 'location'] = ast.literal_eval('[["176 184;201 212"]]')

train.loc[1615, 'annotation'] = ast.literal_eval('[["diarrhea for last 2-3 days"]]')
train.loc[1615, 'location'] = ast.literal_eval('[["249 257;271 288"]]')

train.loc[1664, 'annotation'] = ast.literal_eval('[["no vaginal discharge"]]')
train.loc[1664, 'location'] = ast.literal_eval('[["822 824;907 924"]]')

train.loc[1714, 'annotation'] = ast.literal_eval('[["started about 8-10 hours ago"]]')
train.loc[1714, 'location'] = ast.literal_eval('[["101 129"]]')

train.loc[1929, 'annotation'] = ast.literal_eval('[["no blood in the stool"]]')
train.loc[1929, 'location'] = ast.literal_eval('[["531 539;549 561"]]')

train.loc[2134, 'annotation'] = ast.literal_eval('[["last sexually active 9 months ago"]]')
train.loc[2134, 'location'] = ast.literal_eval('[["540 560;581 593"]]')

train.loc[2191, 'annotation'] = ast.literal_eval('[["right lower quadrant pain"]]')
train.loc[2191, 'location'] = ast.literal_eval('[["32 57"]]')

train.loc[2553, 'annotation'] = ast.literal_eval('[["diarrhoea no blood"]]')
train.loc[2553, 'location'] = ast.literal_eval('[["308 317;376 384"]]')

train.loc[3124, 'annotation'] = ast.literal_eval('[["sweating"]]')
train.loc[3124, 'location'] = ast.literal_eval('[["549 557"]]')

train.loc[3858, 'annotation'] = ast.literal_eval('[["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]]')
train.loc[3858, 'location'] = ast.literal_eval('[["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]')

train.loc[4373, 'annotation'] = ast.literal_eval('[["for 2 months"]]')
train.loc[4373, 'location'] = ast.literal_eval('[["33 45"]]')

train.loc[4763, 'annotation'] = ast.literal_eval('[["35 year old"]]')
train.loc[4763, 'location'] = ast.literal_eval('[["5 16"]]')

train.loc[4782, 'annotation'] = ast.literal_eval('[["darker brown stools"]]')
train.loc[4782, 'location'] = ast.literal_eval('[["175 194"]]')

train.loc[4908, 'annotation'] = ast.literal_eval('[["uncle with peptic ulcer"]]')
train.loc[4908, 'location'] = ast.literal_eval('[["700 723"]]')

train.loc[6016, 'annotation'] = ast.literal_eval('[["difficulty falling asleep"]]')
train.loc[6016, 'location'] = ast.literal_eval('[["225 250"]]')

train.loc[6192, 'annotation'] = ast.literal_eval('[["helps to take care of aging mother and in-laws"]]')
train.loc[6192, 'location'] = ast.literal_eval('[["197 218;236 260"]]')

train.loc[6380, 'annotation'] = ast.literal_eval('[["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]]')
train.loc[6380, 'location'] = ast.literal_eval('[["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]')

train.loc[6562, 'annotation'] = ast.literal_eval('[["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]]')
train.loc[6562, 'location'] = ast.literal_eval('[["290 320;327 337"], ["290 320;342 358"]]')

train.loc[6862, 'annotation'] = ast.literal_eval('[["stressor taking care of many sick family members"]]')
train.loc[6862, 'location'] = ast.literal_eval('[["288 296;324 363"]]')

train.loc[7022, 'annotation'] = ast.literal_eval('[["heart started racing and felt numbness for the 1st time in her finger tips"]]')
train.loc[7022, 'location'] = ast.literal_eval('[["108 182"]]')

train.loc[7422, 'annotation'] = ast.literal_eval('[["first started 5 yrs"]]')
train.loc[7422, 'location'] = ast.literal_eval('[["102 121"]]')

train.loc[8876, 'annotation'] = ast.literal_eval('[["No shortness of breath"]]')
train.loc[8876, 'location'] = ast.literal_eval('[["481 483;533 552"]]')

train.loc[9027, 'annotation'] = ast.literal_eval('[["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]]')
train.loc[9027, 'location'] = ast.literal_eval('[["92 102"], ["123 164"]]')

train.loc[9938, 'annotation'] = ast.literal_eval('[["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]]')
train.loc[9938, 'location'] = ast.literal_eval('[["89 117"], ["122 138"], ["368 402"]]')

train.loc[9973, 'annotation'] = ast.literal_eval('[["gaining 10-15 lbs"]]')
train.loc[9973, 'location'] = ast.literal_eval('[["344 361"]]')

train.loc[10513, 'annotation'] = ast.literal_eval('[["weight gain"], ["gain of 10-16lbs"]]')
train.loc[10513, 'location'] = ast.literal_eval('[["600 611"], ["607 623"]]')

train.loc[11551, 'annotation'] = ast.literal_eval('[["seeing her son knows are not real"]]')
train.loc[11551, 'location'] = ast.literal_eval('[["386 400;443 461"]]')

train.loc[11677, 'annotation'] = ast.literal_eval('[["saw him once in the kitchen after he died"]]')
train.loc[11677, 'location'] = ast.literal_eval('[["160 201"]]')

train.loc[12124, 'annotation'] = ast.literal_eval('[["tried Ambien but it didnt work"]]')
train.loc[12124, 'location'] = ast.literal_eval('[["325 337;349 366"]]')

train.loc[12279, 'annotation'] = ast.literal_eval('[["heard what she described as a party later than evening these things did not actually happen"]]')
train.loc[12279, 'location'] = ast.literal_eval('[["405 459;488 524"]]')

train.loc[12289, 'annotation'] = ast.literal_eval('[["experienced seeing her son at the kitchen table these things did not actually happen"]]')
train.loc[12289, 'location'] = ast.literal_eval('[["353 400;488 524"]]')

train.loc[13238, 'annotation'] = ast.literal_eval('[["SCRACHY THROAT"], ["RUNNY NOSE"]]')
train.loc[13238, 'location'] = ast.literal_eval('[["293 307"], ["321 331"]]')

train.loc[13297, 'annotation'] = ast.literal_eval('[["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]]')
train.loc[13297, 'location'] = ast.literal_eval('[["182 221"], ["182 213;225 234"]]')

train.loc[13299, 'annotation'] = ast.literal_eval('[["yesterday"], ["yesterday"]]')
train.loc[13299, 'location'] = ast.literal_eval('[["79 88"], ["409 418"]]')

train.loc[13845, 'annotation'] = ast.literal_eval('[["headache global"], ["headache throughout her head"]]')
train.loc[13845, 'location'] = ast.literal_eval('[["86 94;230 236"], ["86 94;237 256"]]')

train.loc[14083, 'annotation'] = ast.literal_eval('[["headache generalized in her head"]]')
train.loc[14083, 'location'] = ast.literal_eval('[["56 64;156 179"]]')

In [ ]:
train['annotation_length'] = train['annotation'].apply(len)
display(train['annotation_length'].value_counts())

# CV split

In [ ]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = train['pn_num'].values
for n, (train_index, val_index) in enumerate(Fold.split(train, train['location'], groups)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())
train.head()

In [ ]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

# tokenizer

# Dataset

In [ ]:
# ====================================================
# Define max_len
# ====================================================
for text_col in ['pn_history']:
    pn_history_lengths = []
    tk0 = tqdm(patient_notes[text_col].fillna("").values, total=len(patient_notes))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        pn_history_lengths.append(length)
#     pn_history max(lengths): 433
    LOGGER.info(f'{text_col} max(lengths): {max(pn_history_lengths)}')

for text_col in ['feature_text']:
    features_lengths = []
    tk0 = tqdm(features[text_col].fillna("").values, total=len(features))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        features_lengths.append(length)
    LOGGER.info(f'{text_col} max(lengths): {max(features_lengths)}')
# feature_text max(lengths): 30
# max_len: 466
CFG.max_len = max(pn_history_lengths) + max(features_lengths) + 3 # cls & sep & sep
LOGGER.info(f"max_len: {CFG.max_len}")

In [ ]:
# ====================================================
# Dataset
# ====================================================
# tokenize and standardlize inputs
def prepare_input(cfg, text, feature_text):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


def create_label(cfg, text, annotation_length, location_list):
    encoded = cfg.tokenizer(text,
                            add_special_tokens=True,
                            max_length=CFG.max_len,
                            padding="max_length",
                            return_offsets_mapping=True)
    offset_mapping = encoded['offset_mapping']
    ignore_idxes = np.where(np.array(encoded.sequence_ids()) != 0)[0]
    label = np.zeros(len(offset_mapping))
    label[ignore_idxes] = -1
    if annotation_length != 0:
        for location in location_list:
            for loc in [s.split() for s in location.split(';')]:
                start_idx = -1
                end_idx = -1
                start, end = int(loc[0]), int(loc[1])
                for idx in range(len(offset_mapping)):
                    if (start_idx == -1) & (start < offset_mapping[idx][0]):
                        start_idx = idx - 1
                    if (end_idx == -1) & (end <= offset_mapping[idx][1]):
                        end_idx = idx + 1
                if start_idx == -1:
                    start_idx = end_idx
                if (start_idx != -1) & (end_idx != -1):
                    label[start_idx:end_idx] = 1
    return torch.tensor(label, dtype=torch.float)


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.annotation_lengths = df['annotation_length'].values
        self.locations = df['location'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        label = create_label(self.cfg, 
                             self.pn_historys[item], 
                             self.annotation_lengths[item], 
                             self.locations[item])
        return inputs, label

# Model

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
            
        s = self.config.hidden_size
        print("size###",s)
        self.lstm = nn.LSTM(input_size=768, hidden_size=384,
                            num_layers=2, bidirectional=True, batch_first=True)
        
#         self.lstm = nn.LSTM(self.model.config.hidden_size, self.lstm_hidden_size, bidirectional=True)

        
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
# Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
# - This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).     
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        
        last_hidden_states = outputs[0]
#         enc, _ = self.lstm(last_hidden_states)
        return last_hidden_states
#         with torch.no_grad():
#           embeds, _  = self.model(**inputs)
#         enc, _ = self.lstm(embeds)
#         return enc

    def forward(self, inputs):
        
        feature_outputs = self.model(**inputs)
        feature = feature_outputs[0]
        enc, _ = self.lstm(feature)
        enc = self.fc_dropout(enc)
        output = self.fc(enc)
        return output

# Helpler functions

In [ ]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    #reset training set and vaildation set according to the folds
#     valid_folds:           id  case_num  pn_num  feature_num                                         annotation                                           location                    feature_text                                         pn_history  annotation_length  fold                         location_for_create_labels
# 0  20540_202         2   20540          202                                                 []                                                 []                             IUD  Mrs. Montgomery is a 44yo female presenting wi...                  0     0                                                 []
# 1  43252_400         4   43252          400  [no h/o tremors, no palpitations, no heat into...  [346 360, 346 348;362 374, 346 348;411 427, 80...  Lack-of-other-thyroid-symptoms  45 y/o f with a h/o nervousness since few week...                  4     0  [346 360;346 348;362 374;346 348;411 427;809 8...
# 2  10620_109         1   10620          109                               [decreased appetite]                                          [249 267]             Diminished-appetite  Ms. Powelton is a 20yo F with no sig PMHx who ...                  1     0                                          [249 267]
# 3  01581_003         0    1581            3                               [episodes, episdoes]                                 [105 113, 350 358]           Intermittent-symptoms  Dillon cleaveland is a 17 yo M presenting for ...                  2     0                                  [105 113;350 358]
# 4  83843_800         8   83843          800                           [eating more than usual]                                          [348 370]              Increased-appetite  HO1 67 F presenting with difficulty sleeping p...                  1     0                                          [348 370]

# valid shape:((204, 11),), training shape:((796, 10),) valid_texts shape: (204,)
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)

# valid_text: ['Mrs. Montgomery is a 44yo female presenting with irregular periods for the last 3 years. These range from 2-6 days in length with her cycles lasting from 3 weeks to 4 months. Her menstrual flow can be either heavy or light ranging fro 2-3 pads/day to 5-6 pads/day. Her periods are not painful. She has also had night sweats and hot flashes during this time period. She has not experienced dizziness, palpitations, intermentrual bleeding or vaginal discharge. She is sexually active with her husband (monogamous) and has no dyspurinia but does experience some vaginal dryness which she uses lubrication for. 1 weeks ago did have some nausea and vomitting for a day which resolved on its own. \r\nDenies changes in mood or energy.\r\nPMHx: HTN, takes HCTZ, no surgeries\r\nFMHX: OA, obsesity in mom and HTN in brother'
#  '45 y/o f with a h/o nervousness since few weeks. Patient has recently changed her job from research to lecturing her audience. The nervousness starts suddenly and is more on mondays and sundays. She thinks frequently about her work and performance on mondays. It is however also present throughout the week. It has never happened before. She has no h/o tremors, palpitations, SOB, headaches, sensation of doom, heat intolerance or weight loss. Patient has been eating less due to anxiety. She consumes 5-6 cups of caffeine everyday. She takes tylenol and was hospitalized for delivery. NKDA, she has reached menopause. Patient has no h/o smoking, alcohol consumption 1 drink socially. Husband has had a vasectomy. She is sexually active. \r\nROS- occasional headaches+, No h/o Abdominal pain, nausea, vomiting, bowel or bladder changes, facial pain, sleep changes'
#  'Ms. Powelton is a 20yo F with no sig PMHx who presents to the ED with 8-10 hour of RLQ abdominal pain. Pt describes this pain as dull/alchy, without radiation. Patient does endorse similar episodes ~3/4 times though with less severeity. Pt endorses decreased appetite, diarrhea of 2 days without hematochezia or melena. Pt denies subjective fever, chills, nausea, vomitting. Patient denies dysuria. Patient has 28 day menstrual cycle with 4 days of moderate flow, last one was 2 weeks ago. Patient is currently no on contraceptive, and was last sexually active with a male partner 9 months ago, using condom consistently.\r\nPMHx: Negative, uses ibuprofen for pain currently\r\nPast Surgical Hx: negative\r\nFHx: healthy parents\r\nSHx: does not smoke, drinks EtOH a couple times a month, does not use illicit drugs, has not travelled anywhere recently'
#  'Dillon cleaveland is a 17 yo M presenting for heart pounding. He noticed this 2-3 mo ago. He has had 5-6 episodes of this over that timeframe. No kown triggers, nothing stops them. He gets lightheaded during them. He feels like his heart is bearting really fast " beating out of my chest". They last for 3-4 minutes. Yesterday was the worst of these episdoes. it lasted for 10 minutes, and in additon to the pounding, he has pressure in the middle of his chest that didn\'t radiate, he was also SOB that that time. Those symtpoms resolved after the pounding went away. He does endorse a historyof taking his friend\'s adderall prescription but hx does not coincide with symtpoms. No headache, vision changes, diarrhea, fever, chills, or temperature issues.\r\nPMH: none- born at term, immunizations complete, growth curve normal\r\nmed: adderall\r\nallergiesl none\r\nFH: mom-thyroid problem, dad, heart attack-52\r\nSH: no smoking, alcohol 3-4drinks, drug- MJ'
#  'HO1 67 F presenting with difficulty sleeping past 3 weeks, diufficulty initialting sleep and getting up early. previosuly no similar episodes . son passed away 3 weeks ago and has been notices lack of energy and low mood since visual and auditory hallucinations once since then.lack of interest in daily activities,no change in weight but has been eating more than usual.\r\ndenies snoring.or any urinary or bowl complaints.\r\nros -ve except as above \r\npmhx htn on medication past 15 year s\r\npshx nil \r\nfhx mother alive and well has a history of major depression \r\nmedications lisinopril and hctz \r\nshx non smoker \r\ndrink 2-3 glasses of wine a week cage 0/4 monogamous with hysband retired receptionist excersise regularly till 3 weeks ago']
    valid_texts = valid_folds['pn_history'].values
    #valid_labels [[], [[346, 360], [346, 348], [362, 374], [346, 348], [411, 427], [809, 814], [826, 833], [768, 770]], [[249, 267]], [[105, 113], [350, 358]], [[348, 370]]] length= 204
    valid_labels = create_labels_for_scoring(valid_folds)
        
        
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            # n for name and p for params
            #loop over the model named parameters, if the name of the parameter in no_dacay, add the param to the params list
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters
    
    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    #[{'params': [Parameter containing:
# tensor([[ 0.0067, -0.0069, -0.0080,  ..., -0.0221, -0.0057, -0.0004],
#         [-0.0079,  0.0098, -0.0234,  ..., -0.0141, -0.0073,  0.0069],
#         [-0.0089,  0.0216, -0.0072,  ..., -0.0187, -0.0202,  0.0146],
#         ...,
#         [-0.0033,  0.0059, -0.0197,  ..., -0.0177,  0.0091,  0.0102],
#         [-0.0126, -0.0349, -0.0075,  ..., -0.0281,  0.0034, -0.0225],
#         [ 0.0065, -0.0107, -0.0003,  ..., -0.0150,  0.0082,  0.0031]],
#        device='cuda:0', requires_grad=True), Parameter containing:
# tensor([[-0.1014, -0.0048, -0.0295,  ..., -0.0334,  0.0616,  0.0006],
#         [-0.0174,  0.0850, -0.0525,  ...,  0.0057,  0.0261, -0.0012],
#         [-0.0089,  0.0334, -0.0349,  ...,  0.0391, -0.0177, -0.0127],
#         ...,
#         [ 0.0303,  0.0198, -0.0292,  ...,  0.0812,  0.0044,  0.0423],
#         [ 0.0944, -0.0252, -0.0389,  ..., -0.0740, -0.0243,  0.0389],
#         [ 0.0155, -0.0573, -0.0324,  ..., -0.0103, -0.0305,  0.0228]],
#        device='cuda:0', requires_grad=True), Parameter containing:
    ## for debug
    if fold==0:
#         print('valid_labels',valid_labels[0:5],'length=',len(valid_labels))
#'valid_text:',valid_texts[0:5],'valid_folds:',valid_folds.head(),
        print('valid shape:%s, training shape:%s' %((valid_folds.shape,),(train_folds.shape,)),'valid_texts shape:',valid_texts.shape)
#         print("optimizier params shape:",len(optimizer_parameters))
              
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)
    
    # ====================================================
    # loop
    # ====================================================
    criterion = nn.BCEWithLogitsLoss(reduction="none")
    best_score = 0.
    for epoch in range(CFG.epochs):
        start_time = time.time()
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)
        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        predictions = predictions.reshape((len(valid_folds), CFG.max_len))
        # scoring
        #char_probs.shape=(204,809), 204=test set size, 809=dic size? word size in current pn_history
        char_probs = get_char_probs(valid_texts, predictions, CFG.tokenizer)

        results = get_results(char_probs, th=0.5)
#         preds= [[], [], [[249, 267]], [], [], [[18, 19]], [[625, 633]], [[29, 34], [52, 58], [71, 79]], [[435, 447]], [[235, 249], [251, 253], [257, 260]], [[1, 5]], [[294, 305]], [], [[13, 31]], [[831, 846]], [[202, 211], [223, 228]], [], [[58, 77]], [], [[356, 360], [367, 370]], [], [], [], [[85, 90], [122, 127], [135, 155]], [], [[413, 422]], [], [[81, 90]], [[10, 15]], [[877, 893]], [], [[17, 25]], [[242, 248], [262, 266], [307, 316]], [], [], [[56, 77]], [[657, 672]], [[502, 511], [514, 517]], [], [[139, 150]], [], [[266, 272], [277, 285]], [[689, 697]], [[84, 116], [685, 689]], [[649, 650], [655, 681]], [[447, 455], [460, 465]], [[395, 397], [399, 406]], [[434, 437]], [[338, 354]], [[339, 356], [422, 433], [451, 459]], [], [[443, 457]], [[35, 45]], [], [[227, 235]], [[845, 847], [855, 861]], [[182, 205]], [], [], [[331, 346]], [[240, 254]], [[343, 345]], [[45, 61], [71, 85], [98, 104]], [], [], [[252, 255], [257, 273]], [], [[61, 68]], [[368, 392]], [[491, 502], [515, 519]], [], [], [], [], [], [[53, 56]], [], [], [], [[488, 510], [542, 547]], [[360, 378]], [[326, 328], [617, 623], [633, 638], [640, 658]], [], [[81, 93], [141, 149]], [[418, 422]], [], [], [], [], [], [[242, 249], [253, 266]], [[249, 264]], [], [[299, 301]], [[365, 369]], [], [], [[21, 27]], [[76, 80], [85, 99], [123, 151]], [[202, 213], [275, 287]], [], [[250, 263]], [[165, 183]], [[1, 4]], [], [[5, 17]], [], [[533, 552]], [], [[300, 306]], [[266, 273]], [], [[30, 39]], [], [[200, 205], [209, 214], [414, 421], [426, 434], [469, 481]], [[384, 387]], [[1, 7]], [], [[15, 25]], [[27, 42], [56, 64]], [[292, 303]], [], [[346, 362], [364, 397]], [[33, 49]], [[258, 262], [291, 297]], [[561, 591]], [[300, 331]], [], [[154, 164], [168, 182]], [], [[15, 26]], [[281, 286], [291, 296], [447, 459]], [[540, 565]], [], [], [[125, 136]], [[351, 354], [366, 388]], [[76, 90], [96, 116]], [[359, 378]], [[50, 65]], [[276, 279], [282, 284], [292, 299]], [], [[165, 175], [223, 240]], [], [], [], [], [], [], [[33, 43]], [[828, 838]], [[79, 93]], [], [[351, 354]], [[250, 258]], [[288, 293]], [[631, 632]], [], [[66, 74], [80, 92], [110, 114], [340, 348]], [[16, 27]], [], [], [[24, 29]], [[73, 97], [200, 216]], [], [[525, 536]], [[43, 58]], [], [[1, 5]], [], [[110, 113], [115, 131]], [], [[251, 266]], [[292, 300]], [[363, 372], [377, 388]], [[284, 303]], [], [[65, 80]], [[483, 488], [516, 529], [544, 547]], [], [[125, 133]], [], [], [], [[8, 18]], [[308, 319]], [[44, 58], [70, 74], [81, 84]], [], [], [[134, 142]], [[15, 20]], [[178, 193]], [[84, 101]], [], [], [[113, 129]], [[137, 147], [163, 168]], [], [[199, 219]], [], [], [[416, 423], [459, 467]], [[5, 9]], []] 
#         shape=(204,)
        preds = get_predictions(results)
        score = get_score(valid_labels, preds)
#         if epoch==0 and fold==0:
#             tensor_char_probs=torch.tensor(char_probs)
#             print(char_probs)
#             print('char_probs.shape=(%d,%d)'%(len(char_probs),len(char_probs[0])))
            
        elapsed = time.time() - start_time
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score < score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
    #load best result model
    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    #[i for i in range()] means [0,1,2,...,CFG.max_len]
    #CFG.max_len=512
    valid_folds[[i for i in range(CFG.max_len)]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        #load vaildation data to gpu
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        loss = torch.masked_select(loss, labels.view(-1, 1) != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = create_labels_for_scoring(oof_df)
        predictions = oof_df[[i for i in range(CFG.max_len)]].values
        char_probs = get_char_probs(oof_df['pn_history'].values, predictions, CFG.tokenizer)
        results = get_results(char_probs, th=0.5)
        preds = get_predictions(results)
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        #    n_fold=5
        #    trn_fold=[0, 1, 2, 3, 4]
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
#train.head() is like:                 
#id	case_num	pn_num	feature_num	annotation	location	feature_text	pn_history-> fold
# 0	00016_000	0	16	0	[dad with recent heart attcak]	[696 724]	Family-history-of-MI-OR-Family-history-of-myoc...	HPI: 17yo M presents with palpitations. Patien...
# 1	00016_001	0	16	1	[mom with "thyroid disease]	[668 693]	Family-history-of-thyroid-disorder	HPI: 17yo M presents with palpitations. Patien...
# 2	00016_002	0	16	2	[chest pressure]	[203 217]	Chest-pressure	HPI: 17yo M presents with palpitations. Patien...
# 3	00016_003	0	16	3	[intermittent episodes, episode]	[70 91, 176 183]	Intermittent-symptoms	HPI: 17yo M presents with palpitations. Patien...
# 4	00016_004	0	16	4	[felt as if he were going to pass out]	[222 258]	Lightheaded	HPI: 17yo M presents with palpitations. Patien...
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
#         print("oof_df shape="+oof_df.shape)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
#         oof_df.to_csv(OUTPUT_DIR+'submission.csv',index=False)
#         model.predcit 
    if CFG.wandb:
        wandb.finish()
        

In [ ]:
# pred,pred.shape,labels
# # with 
# !cd ../input/nbme-deberta-base-baseline-train/